# Carbon Nanotubes Data Set
https://archive.ics.uci.edu/ml/datasets/Carbon+Nanotubes

In [44]:
import mysql.connector as connection
import csv
import logging as log

class CarbonDatatubes:
    #carbon_nanotubes.csv file is hardcoded and available within curren directory
    def __init__(self,host,user,passwd):
        self.host = host
        self.user = user
        self.passwd = passwd
        self.file_src = "carbon_nanotubes.csv"
        log.basicConfig(filename='CarbonDatatubesLogging.log', level=log.INFO,
                        format='%(asctime)s | %(levelname)s | %(message)s')
        
    #MySQL Connection
    def establish_connection(self,database=''):
        return connection.connect(host=self.host, user=self.user, database=database,
                                  passwd=self.passwd,use_pure=True)
        
    #New Database creation in MySQL DB
    def create_database(self,database):
        '''
        Creates database in MySQL.
        Parameters:
            database (str): database name
        Returns:
            N/A
        '''
        try :
            self.database = database
            mydb = self.establish_connection()
            mycursor = mydb.cursor()
            mycursor.execute("CREATE DATABASE IF NOT EXISTS "+database)
            log.info("Database created!")
        except Exception as e:
            log.error("Exception occurred : "+str(e), exc_info=True)
        finally :
            mydb.close()
            
    #New table creation in MySQL DB
    def create_table(self,table):
        '''
        Creates table in MySQL database.
        Parameters:
            table (str): table name
        Returns:
            N/A
        '''
        try :
            self.table = table
            mydb = self.establish_connection(self.database)
            mycursor = mydb.cursor()
            columns = self.get_column_names()
            query = "CREATE TABLE IF NOT EXISTS "+table+" ("+columns+")"
            mycursor.execute(query)
            log.info("Table created!")
        except Exception as e:
            log.error("Exception occurred : "+str(e), exc_info=True)
        finally :
            mydb.close()
            
    #Get header of csv to define as columns dynamically in MySQL table
    def get_column_names(self):
        '''
        Defines columns for table in MySQL database.
        Parameters:
            N/A
        Returns:
            columns (str): comma-separated under-scored column names from csv file header row
        '''
        columns = ""
        with open(self.file_src,'r') as data:
            data_csv = csv.reader(data,delimiter='\n')
            for i in data_csv:
                columns = str(i[0]).replace(" ","_")
                break
            columns = columns.replace("'","")
            #All columns are generically considered string - VARCHAR in database
            columns = columns.replace(";"," VARCHAR(40),")
            columns += " VARCHAR(40)"
        return columns
    
    def read_data(self):
        '''
        Read csv file and put records in table of MySQL database.
        Parameters:
            N/A
        Returns:
            N/A
        '''
        try :
            mydb = self.establish_connection(self.database)
            mycursor = mydb.cursor()
            with open(self.file_src,'r') as data:
                next(data)
                data_csv = csv.reader(data,delimiter='\n')
                for i in data_csv:
                    values = ""
                    record = str(i[0])
                    records = record.split(",")
                    #Cases where last column value is missing
                    if len(records) == 6:
                        records.append("")
                    #Read valuees within each row individually
                    for j in range(len(records)):
                        #Case for 1st column in csv file split
                        #in 2 values, for 1st 2 columns of MySQL table 
                        if j==0:
                            initials = str(records[j]).split(";")
                            values += "'"+initials[0]+"','"+initials[1]+"'"
                        else:
                            values += ",'"+str(records[j]).replace(';',',')+"'"
                    mycursor.execute("INSERT INTO "+self.table+" VALUES ("+values+")")
                    mydb.commit()
            log.info("Records created!")
        except Exception as e:
            log.error("Exception occurred : "+str(e), exc_info=True)
        finally :
            mydb.close()
            log.shutdown()

cd = CarbonDatatubes("localhost","root","MySQL@2021") #Local MySQL server details
cd.create_database("CarbonDatatubes") #Database name
cd.create_table("CarbonTubes") #Table name
cd.read_data() #Main function o read file